In [1]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
df = pd.read_csv('FBGM_League_1_all_seasons_Average_Stats(1).csv')
df.shape
#df = df.sample(10000)
df

,pid,Name,Pos,DraftPick,Age,Salary,Team,Season,G,GS,...,RBk,PBk,PCv,Tck.1,PRs,RnS,KPw,KAc,PPw,PAc
0,0,Jeremis Tolbert,QB,24.0,27,30.00,PHO,2019,16,16,...,22,25,16,15,16,14,20,16,12,16
1,1,Doyle Johnson,OL,25.0,28,19.65,MXC,2019,14,14,...,84,88,28,13,15,16,22,18,20,17
2,4,Tyler Austin,OL,3.0,24,5.95,SAC,2019,14,14,...,83,74,15,10,10,10,13,16,9,17
3,5,Aubrey Houston,TE,4.0,25,5.15,SA,2019,14,14,...,76,10,25,21,13,9,15,13,14,10
4,6,Rodney Reed,OL,21.0,24,1.30,SAC,2019,8,8,...,73,75,19,14,10,10,15,71,16,11
5,7,Jake Williams,DL,9.0,22,2.85,PHI,2019,15,15,...,36,77,25,18,75,79,17,8,22,20
6,8,Lamar Chatmon,QB,3.0,25,5.95,VAN,2019,16,16,...,18,20,29,5,16,5,3,8,3,10
7,9,Chris Turner,OL,9.0,24,2.85,SEA,2019,15,15,...,70,73,11,10,10,8,14,46,9,17
8,10,Troy Haynes,S,13.0,22,2.20,MXC,2019,16,16,...,19,17,70,57,12,75,14,17,12,9
9,11,Sam Fruendt,DL,23.0,23,1.30,BAL,2019,16,16,...,28,8,19,12,71,88,9,12,12,18


In [3]:
df.columns

Index(['pid', 'Name', 'Pos', 'DraftPick', 'Age', 'Salary', 'Team', 'Season',
       'G', 'GS',
       ...
       'RBk', 'PBk', 'PCv', 'Tck.1', 'PRs', 'RnS', 'KPw', 'KAc', 'PPw', 'PAc'],
      dtype='object', length=126)

In [4]:
y = df.iloc[:,105:]
X = df.iloc[:,11:103]
#y = y[(X['AST%'] >0) & (X['AST%'] < 100)]
print(X.shape)
#X = X[(X['AST%'] >0) & (X['AST%'] < 100)]
print(X.shape)

valid_col = [_ for _ in X.columns if not _ in [ 'FL', 'Pen', 'Yds.6','Lng.4', 'APY', 'AV']]
X = X[valid_col]
replacement_filter = (df.Salary >= 0.25) & (df.Salary < 1.0)
replacement_player_mean_bs = X[replacement_filter].mean()
replacement_player_std_bs = X[replacement_filter].std()
replacement_player_cov_bs = X[replacement_filter].cov()

replacement_player_mean_r = y[replacement_filter].mean()
replacement_player_std_r = y[replacement_filter].std()
replacement_player_cov_r = y[replacement_filter].cov()


replacement_player_mean_r

X.columns
#valid_col

(18051, 92)
(18051, 92)


Index(['Cmp', 'Att', 'Pct', 'Yds', 'TD', 'TD%', 'Int', 'Int%', 'Lng', 'Y/A',
       'AY/A', 'Y/C', 'Y/G', 'QBRat', 'Sk', 'Yds.1', 'NY/A', 'ANY/A', 'Sk%',
       'Rush', 'Yds.2', 'TD.1', 'Lng.1', 'Y/A.1', 'Y/G.1', 'A/G', 'Tgt', 'Rec',
       'Yds.3', 'TD.2', 'Lng.2', 'Y/R', 'R/G', 'Y/G.2', 'Ctch%', 'Tch',
       'Y/Tch', 'YScm', 'RRTD', 'Int.1', 'Yds.4', 'TD.3', 'Lng.3', 'PD', 'FF',
       'FR', 'Yds.5', 'TD.4', 'Sk.1', 'Tck', 'Solo', 'Ast', 'TFL', 'Sfty',
       'Fmb', 'FG10', 'FGA10', 'FG20', 'FGA20', 'FG30', 'FGA30', 'FG40',
       'FGA40', 'FG50', 'FGA50', 'FGM', 'FGA', 'Pct.1', 'XPM', 'XPA', 'Pct.2',
       'Pnt', 'Yds.7', 'Lng.5', 'Blk', 'Y/A.2', 'PR', 'Yds.8', 'TD.5', 'Lng.6',
       'Y/A.3', 'KR', 'Yds.9', 'TD.6', 'Lng.7', 'Y/A.4'],
      dtype='object')

In [5]:
from sklearn import neural_network
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import feature_selection, model_selection
from sklearn import  multioutput
from sklearn import ensemble
from sklearn import svm

In [6]:
fexp = preprocessing.PolynomialFeatures(degree=1,interaction_only=False)
scalerX = preprocessing.RobustScaler()
scalery = preprocessing.StandardScaler()
prescale_X = scalerX.fit_transform(X)
prescale_y = scalery.fit_transform(y)
prescale_X = fexp.fit_transform(prescale_X)
X_train, X_test, y_train, y_test = model_selection.train_test_split(prescale_X, prescale_y, test_size=0.25, random_state=42)

/Users/lkeselma/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/lkeselma/anaconda/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [7]:
trials = 1
ts = []
for i in range(trials):
    #clf = neural_network.MLPRegressor((36,5,24,36),'tanh',solver='adam',max_iter=1000)
    #clf = neural_network.MLPRegressor((),'identity',solver='lbfgs',alpha=5e2,tol=1e-9)
    #clf = multioutput.MultiOutputRegressor(linear_model.SGDRegressor(penalty='l2',alpha=5e2,eta0=1e-6,tol=1e-12,max_iter=50,verbose=True))
    clf = multioutput.MultiOutputRegressor(linear_model.ElasticNet(alpha=5e-3,l1_ratio=0.5))
    #clf = ensemble.ExtraTreesRegressor(8,criterion='mae',max_depth=3,verbose=1)
    #clf = multioutput.MultiOutputRegressor(svm.SVR())
    #clf.fit(X_train,y_train)
    #print(clf.score(X_train,y_train),clf.score(X_test,y_test))
    clf.fit(prescale_X,prescale_y)
    yt = scalery.inverse_transform(clf.predict(prescale_X))
    err = np.linalg.norm(yt-y)
    ts.append((err,clf))

/Users/lkeselma/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/lkeselma/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/lkeselma/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/lkeselma/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective di

In [8]:
ts = sorted(ts)[::1] # why not the biggest error
print(ts[0][0])
clf = ts[0][1]
clf.score(prescale_X,prescale_y)

8567.612406894588


0.4157314434021972

In [9]:
col_names = X.columns
col_names = fexp.get_feature_names(X.columns)

for i,c in enumerate(y.columns):
    coeffs = clf.estimators_[i].coef_ 
    v = np.argsort(abs(coeffs))[::-1]
    print(c)
    coeffs2 = [(coeffs[i2],col_names[i2]) for i2 in v[:10]]
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('{:.2f} * {} + '.format(v,n),end='')
    print('| Variable | Coeff |')
    print('|----------|-------|')
    for v,n in sorted(coeffs2,reverse=True):
        print('|{:25s}|{:.2f}|'.format(n,v))
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('\t{:25s}\t{:.2f}'.format(n,v))


Hgt
| Variable | Coeff |
|----------|-------|
|Sk.1                     |0.17|
|FR                       |0.05|
|Y/Tch                    |0.04|
|Pnt                      |0.01|
|Cmp                      |0.01|
|Y/R                      |-0.03|
|FGM                      |-0.03|
|Y/A.1                    |-0.04|
|Int.1                    |-0.05|
|PD                       |-0.11|
Str
| Variable | Coeff |
|----------|-------|
|Solo                     |0.21|
|FR                       |0.17|
|Tck                      |0.17|
|Ast                      |0.11|
|Sk.1                     |0.07|
|FF                       |-0.03|
|Y/A.1                    |-0.03|
|Int.1                    |-0.05|
|TFL                      |-0.13|
|PD                       |-0.15|
Spd
| Variable | Coeff |
|----------|-------|
|PD                       |0.12|
|Int.1                    |0.10|
|Y/A.1                    |0.06|
|TFL                      |0.03|
|Solo                     |0.03|
|Y/R                      |

In [10]:
GEN_YEAR = 2018
tyear = [GEN_YEAR]
#if tyear[0] < 1980:
#    print("MY PARSING OF THE TABLES IS WRONG WITHOUT the 2PA/3PA TRACKS")
#    raise

CURRENT_YEAR = 2018
all_tables = {}
for ty in tyear: 
    all_tables[ty] = np.load('fb_tables_{}.pkl'.format(ty))
teams = all_tables[tyear[0]].keys()


In [11]:
player_stats = {k:{} for k in tyear}
table_columns = defaultdict(dict)
for ty in tyear:
    tables = all_tables[ty]
    for team in tables:
        team_tables =  tables[team]
        for table_name in team_tables:
            if table_name in ['draft-rights','team_and_opponent','conf','name','logo']:
                continue
            table = team_tables[table_name].fillna(0)

            #print(table_name)
            #print(table.index)
            for row in table.itertuples():
                name = row[0]
                name = name.replace('\xa0\xa0',' ')
                if name == 'Team Totals':
                    continue
                nsplit = name.split(' ')
                if nsplit[-1] in ['Jr.','Sr.','I','II','III',"IV",'(TW)']:
                    name = ' '.join(nsplit[:-1])

                player_table = player_stats[ty].get(name,{})
                player_row = player_table.get(table_name,[])
                player_row = player_row + [row]

                player_table[table_name] = player_row
                player_table['team'] = team
                player_stats[ty][name] = player_table
                
                #if name == 'Dennis Smith Jr.' or name == 'Luka Doncic':
                #    print(player_stats[ty][name],team)
            tcs = list(table.columns)
            for i in range(len(tcs)):
                while tcs.index(tcs[i]) != i:
                    if '.' in tcs[i]:
                        tsplit = tcs[i].split('.')
                        tsplit[-1] = str(int(tsplit[-1]) + 1)
                        tcs[i] = '.'.join(tsplit)
                    else:
                        tcs[i] += '.1'
            table_columns[(ty,team)][table_name] = tcs

In [12]:
table_mask = defaultdict(dict)
for key in table_columns:
    for table in table_columns[key]:
        table_mask[key][table] = [_.strip() !=''  for _ in table_columns[key][table] ]
        table_columns[key][table] = [_ for _ in table_columns[key][table] if _.strip() != '']
#for player in player_stats:
#    for table_in in player_stats[player]:
#        if 'on_off' in table_in or 'salaries' in table_in:
#            continue
#        if len(player_stats[player][table_in]) > 1:
#            pass
            #print(player,table_in,'MP' in player_stats[player][table_in][0]._fields)
            #print(player_stats[player][table_in][0])

In [13]:
# add playoff data to normal data
if False:
    for ty in tyear:
        for player in player_stats[ty]:
            for table_in in player_stats[ty][player]:
                tableN = table_in.split('_')
                tableS = '_'.join(tableN[1:])
                if 'playoffs'==tableN[0] and not table_in in ['playoffs_pbp']:
                    #print(table_in)
                    if tableS in player_stats[ty][player]:
                        player_stats[ty][player][tableS] += player_stats[ty][player][table_in]

In [14]:
#player_stats[1968]['Bill Allen']
#player_stats[1968]['Jim Burns']['per_game'], table_columns[(1968,'DLC')]['per_game'].index('MP')

In [15]:
for ty in tyear:
    for player in player_stats[ty]:
        team = player_stats[ty][player]['team']
        for tt in player_stats[ty][player]:
            if tt == 'team':
                continue
            new_rows = []
            for tablet in player_stats[ty][player][tt]:
                vector = [_ if _ != '' else '0.0' for _ in tablet[1:]]
                vector = [(float(_.replace('%',''))/100 if type(_) == str and'%' in _ else _) for _ in vector]
                if 'on_off' in tt:
                    vector = vector[1:]
                if 'contracts' in tt:
                    vector = vector[1:-2]
                if tt in ['salaries2','contracts']:
                    vector = [_.replace(',','').replace('$','') for _ in vector]
                try:
                    v2 = np.array(vector).astype(np.float)
                except:
                    v2 = vector
                new_rows.append(vector)
            a = np.array(new_rows)


            if tt in table_columns[(ty,team)]:
                if 'MP' in table_columns[(ty,team)][tt] and not tt in ['pbp','on_off','on_off_p']:
                    try:
                        a = a.astype(np.float)
                    except:
                        a = list(a)
                        a[0] = np.array([float(_) for _ in a[0]])
                        a[1] = np.array([float(_) for _ in a[1]])
                        a = np.array(a)
                    try:
                        mins = a[:,table_columns[(ty,team)][tt].index('MP')].reshape((-1,1))
                        new_rows = ((a.T @ mins)/mins.sum()).T
                        a = new_rows
                    except:
                        #print(tt,a.shape,player,a,mins)
                        #print('.',end='')
                        
                        print(ty,team,player,tt)
                        continue
            player_stats[ty][player][tt] = a


In [16]:
player_stats

{2018: {'Matt Ryan': {'passing': array([['33.0', 'QB', '16', '16.0', '7-9-0', '422', '608', '69.4',
           '4924', '35', '5.8', '7', '1.2', '75.0', '8.1', '8.7', '11.7',
           '307.8', '108.1', '70.6', '42', '296', '7.12', '7.71', '6.5',
           '1.0', '1.0']], dtype='<U32'),
   'team': 'atl',
   'rushing_and_receiving': array([['33.0', 'QB', '16', '16.0', '33', '125', '3', '15.0', '3.8',
           '7.8', '2.1', '1.0', '1.0', '5.0', '5.0', '1.0', '5.0', '0.1',
           '0.3', '1.0', '5.0', '34.0', '3.8', '130', '4', '10']],
         dtype='<U32'),
   'defense': array([['33.0', 'QB', '16', '16.0', '0.0', '0.0', '0.0', '0.0', '0.0',
           '0.0', '10.0', '2.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
           '0.0', '0.0', '0.0']], dtype='<U32'),
   'scoring': array([['33.0', 'QB', '16', '16.0', '3.0', '1.0', '0.0', '0.0', '0.0',
           '0.0', '0.0', '4.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
           '0.0', '24', '1.5']], dtype='<U32')},
  'Matt Schaub

In [17]:
len(X.columns)

86

In [18]:
X.columns

Index(['Cmp', 'Att', 'Pct', 'Yds', 'TD', 'TD%', 'Int', 'Int%', 'Lng', 'Y/A',
       'AY/A', 'Y/C', 'Y/G', 'QBRat', 'Sk', 'Yds.1', 'NY/A', 'ANY/A', 'Sk%',
       'Rush', 'Yds.2', 'TD.1', 'Lng.1', 'Y/A.1', 'Y/G.1', 'A/G', 'Tgt', 'Rec',
       'Yds.3', 'TD.2', 'Lng.2', 'Y/R', 'R/G', 'Y/G.2', 'Ctch%', 'Tch',
       'Y/Tch', 'YScm', 'RRTD', 'Int.1', 'Yds.4', 'TD.3', 'Lng.3', 'PD', 'FF',
       'FR', 'Yds.5', 'TD.4', 'Sk.1', 'Tck', 'Solo', 'Ast', 'TFL', 'Sfty',
       'Fmb', 'FG10', 'FGA10', 'FG20', 'FGA20', 'FG30', 'FGA30', 'FG40',
       'FGA40', 'FG50', 'FGA50', 'FGM', 'FGA', 'Pct.1', 'XPM', 'XPA', 'Pct.2',
       'Pnt', 'Yds.7', 'Lng.5', 'Blk', 'Y/A.2', 'PR', 'Yds.8', 'TD.5', 'Lng.6',
       'Y/A.3', 'KR', 'Yds.9', 'TD.6', 'Lng.7', 'Y/A.4'],
      dtype='object')

In [19]:
#'returns', 'kicking'
table_columns[(2018,'nwe')]['returns']


['Age',
 'Pos',
 'G',
 'GS',
 'Ret',
 'Yds',
 'TD',
 'Lng',
 'Y/R',
 'Rt',
 'Yds.1',
 'TD.1',
 'Lng.1',
 'Y/Rt',
 'APYd']

In [20]:
lt = [
      ('Cmp','passing'),('Att','passing'),('Pct','passing','Cmp%'),
      ('Yds','passing'),('TD','passing'),('TD%','passing'),('Int','passing'),
      ('Int%','passing'),('Lng','passing'),('Y/A','passing'),('AY/A','passing'),('QBRat','passing','Rate'),
      ('Y/C','passing'),('Y/G','passing'),('Sk','passing'),('Yds.1','passing'),('NY/A','passing'),
      ('ANY/A','passing'),('Sk%','passing'),('Rush','rushing_and_receiving','Att'),
      ('Yds.2','rushing_and_receiving','Yds'),('TD.1','rushing_and_receiving','TD'),('Lng.1','rushing_and_receiving','Lng'),
      ('Y/A.1','rushing_and_receiving','Y/A'),('Y/G.1','rushing_and_receiving','Y/G'),('A/G','rushing_and_receiving'),
      ('Tgt','rushing_and_receiving'),('Rec','rushing_and_receiving'),('Yds.3','rushing_and_receiving','Yds.1'),
      ('TD.2','rushing_and_receiving','TD.1'),('Lng.2','rushing_and_receiving','Lng.1'),('Y/R','rushing_and_receiving'),
      ('R/G','rushing_and_receiving'),('Y/G.2','rushing_and_receiving','Y/G.1'),('Ctch%','rushing_and_receiving'),
      ('Y/Tch','rushing_and_receiving'),('Y/Tgt','rushing_and_receiving'),('Tch','rushing_and_receiving','Touch'),
      ('YScm','rushing_and_receiving'),('RRTD','rushing_and_receiving'),('Int.1','defense','Int'),
      ('Yds.4','defense','Yds'),('TD.3','defense','TD'),('Lng.3','defense','Lng'),('PD','defense'),
      ('FF','defense'),('FR','defense'),('Yds.5','defense','Yds.1'),('TD.4','defense','TD.1'),('Sk.1','defense','Sk'),
      ('Tck','defense','Comb'),('Solo','defense'),('Ast','defense'),('TFL','defense'),('Sfty','defense'),('Fmb','defense'),
      ('FG10','kicking','FGM'),('FGA10','kicking','FGA'),('FG20','kicking','FGM.1'),('FGA20','kicking','FGA.1'),
      ('FG30','kicking','FGM.2'),('FGA30','kicking','FGA.2'),('FG40','kicking','FGM.3'),('FGA40','kicking','FGA.3'),
      ('FG50','kicking','FGM.4'),('FGA50','kicking','FGA.4'),('FGM','kicking','FGM.5'),('FGA','kicking','FGA.5'),
      ('Pct.1','kicking','FG%'),('XPM','kicking'),('XPA','kicking'),('Pct.2','kicking','XP%'),('Pnt','kicking'),
      ('Yds.7','kicking','Yds'),('Lng.5','kicking','Lng'),('Blk','kicking','Blck'),('Y/A.2','kicking','Y/P'),
      ('PR','returns','Ret'),('Yds.8','returns','Yds'),('TD.5','returns','TD'),('Lng.6','returns','Lng'),
      ('Y/A.3','returns','Y/R'),('KR','returns','Rt'),('Yds.9','returns','Yds.1'),('TD.6','returns','TD.1'),
      ('Lng.7','returns','Lng.1'),('Y/A.4','returns','Y/Rt')]

locations = []
for loc in lt:
    if len(loc) == 2:
        loc = list(loc)
        loc.append(loc[0])
    locations.append(loc)

        
locs = defaultdict(dict)
for ty in tyear:
    for team in all_tables[ty]:
        for loc in locations:
            try:
                locs[(ty,team)][loc[0]] = (table_columns[(ty,team)][loc[1]].index(loc[2]),loc[1])
            except Exception as e:
                #print(e)
                #print(team,ty)
                print(e,loc)
                pass#print(loc,table_columns[loc[1]])
    #if 'shooting' in table_columns:
    #    l = list(locs['LowPostFGP'])
    #    l[0] +=2
    #    locs['MidRangeFGP'] = l
len(locs[(2018,'nwe')]),locs[(2018,'nwe')]

(87,
 {'Cmp': (5, 'passing'),
  'Att': (6, 'passing'),
  'Pct': (7, 'passing'),
  'Yds': (8, 'passing'),
  'TD': (9, 'passing'),
  'TD%': (10, 'passing'),
  'Int': (11, 'passing'),
  'Int%': (12, 'passing'),
  'Lng': (13, 'passing'),
  'Y/A': (14, 'passing'),
  'AY/A': (15, 'passing'),
  'QBRat': (18, 'passing'),
  'Y/C': (16, 'passing'),
  'Y/G': (17, 'passing'),
  'Sk': (20, 'passing'),
  'Yds.1': (21, 'passing'),
  'NY/A': (22, 'passing'),
  'ANY/A': (23, 'passing'),
  'Sk%': (24, 'passing'),
  'Rush': (4, 'rushing_and_receiving'),
  'Yds.2': (5, 'rushing_and_receiving'),
  'TD.1': (6, 'rushing_and_receiving'),
  'Lng.1': (7, 'rushing_and_receiving'),
  'Y/A.1': (8, 'rushing_and_receiving'),
  'Y/G.1': (9, 'rushing_and_receiving'),
  'A/G': (10, 'rushing_and_receiving'),
  'Tgt': (11, 'rushing_and_receiving'),
  'Rec': (12, 'rushing_and_receiving'),
  'Yds.3': (13, 'rushing_and_receiving'),
  'TD.2': (15, 'rushing_and_receiving'),
  'Lng.2': (16, 'rushing_and_receiving'),
  'Y/R': (

In [21]:
player_vectors = []
player_names = []
player_years = []
player_scales = []
player_heights = []
player_ages = []
player_teams = []
player_pos = []
for ty in tyear:
    for name in player_stats[ty]:
        try:
            stats = player_stats[ty][name]
            team = stats['team']

            d = {}
            for k,v in locs[(ty,team)].items():
                SV  = 0
                try:
                    SV = float(stats[v[1]][0][v[0]])
                except:
                    pass
                d[k] = 0 if SV == '' else SV
            d['Pos'] = []
            for t in ['passing','defense','returns','kicking','rushing_and_receiving']:
                try:
                    d['Age'] = int(float(stats[t][0][table_columns[(ty,team)][t].index('Age')]))
                except:
                    pass
                try:
                    d['Pos'].append(stats[t][0][table_columns[(ty,team)][t].index('Pos')])
                except:
                    pass
            player_vectors.append([d[stat] for stat in X.columns])
            player_names.append(name)
            player_years.append(ty)
            player_ages.append(d['Age'])
            player_pos.append(d['Pos'])

            player_teams.append(team)
        except Exception as e:
            print(name,team)
            print(e)
            print(d.keys())
            #if name == 'Bill Bradley':
            #    raise
            raise
X.columns

Index(['Cmp', 'Att', 'Pct', 'Yds', 'TD', 'TD%', 'Int', 'Int%', 'Lng', 'Y/A',
       'AY/A', 'Y/C', 'Y/G', 'QBRat', 'Sk', 'Yds.1', 'NY/A', 'ANY/A', 'Sk%',
       'Rush', 'Yds.2', 'TD.1', 'Lng.1', 'Y/A.1', 'Y/G.1', 'A/G', 'Tgt', 'Rec',
       'Yds.3', 'TD.2', 'Lng.2', 'Y/R', 'R/G', 'Y/G.2', 'Ctch%', 'Tch',
       'Y/Tch', 'YScm', 'RRTD', 'Int.1', 'Yds.4', 'TD.3', 'Lng.3', 'PD', 'FF',
       'FR', 'Yds.5', 'TD.4', 'Sk.1', 'Tck', 'Solo', 'Ast', 'TFL', 'Sfty',
       'Fmb', 'FG10', 'FGA10', 'FG20', 'FGA20', 'FG30', 'FGA30', 'FG40',
       'FGA40', 'FG50', 'FGA50', 'FGM', 'FGA', 'Pct.1', 'XPM', 'XPA', 'Pct.2',
       'Pnt', 'Yds.7', 'Lng.5', 'Blk', 'Y/A.2', 'PR', 'Yds.8', 'TD.5', 'Lng.6',
       'Y/A.3', 'KR', 'Yds.9', 'TD.6', 'Lng.7', 'Y/A.4'],
      dtype='object')

In [22]:
first_n = len([yr for yr in player_years if yr == tyear[0]])
gen_FA = len(teams)*5 if CURRENT_YEAR!=GEN_YEAR else 0
first_n,len(teams),gen_FA
#player_names

(1673, 32, 0)

In [23]:
Xn = np.nan_to_num(np.array(player_vectors))
#Xn.shape,Xn_s.shape,prescale_X.shape,fexp.n_input_features_,X.shape

In [24]:
#Xn = np.nan_to_num(np.array(player_vectors))
# tuned this to get roughly 8-12 players at 70 or above. Which seemed like normal for a league
scalerX2 = preprocessing.RobustScaler()#(quantile_range=(30.0, 70.0))
scalerX2.fit(Xn[:first_n])
#scalerX2 = scalerX
Xn_s =scalerX2.transform(np.nan_to_num(Xn))
Xn_fs = fexp.transform(np.nan_to_num(Xn_s))
predict = clf.predict(Xn_fs)
ratings = np.nan_to_num(scalery.inverse_transform(predict))


In [25]:
v = X.mean(0)-Xn.mean(0)
v.sort_values()

Yds     -12.678081
Yds.3   -11.171805
YScm     -9.781420
QBRat    -2.742722
Pct      -1.919220
Y/G.2    -1.343084
Yds.4    -1.314263
Y/G      -1.107088
Cmp      -1.094266
Rec      -0.980100
Lng.3    -0.908621
Lng.7    -0.896562
Y/Tch    -0.874137
Y/A.4    -0.792647
Y/R      -0.717124
TD%      -0.711711
Lng.2    -0.676318
AY/A     -0.458974
ANY/A    -0.450411
Lng      -0.446849
Y/C      -0.384599
Att      -0.360706
Y/A      -0.355249
NY/A     -0.345600
Y/A.1    -0.126446
R/G      -0.124338
Sk%      -0.111582
RRTD     -0.098310
Lng.1    -0.090782
Int%     -0.086188
           ...    
Int       0.009572
FR        0.009903
FGA40     0.010521
TD.5      0.014418
Int.1     0.016745
Sfty      0.030816
Y/A.2     0.117690
FF        0.125265
Y/A.3     0.132672
Lng.5     0.163106
KR        0.167582
PD        0.291600
Rush      0.456799
Tgt       0.488068
Pnt       0.493486
Yds.1     0.812622
Lng.6     0.830197
PR        1.052095
Yds.5     1.252698
Yds.2     1.390386
Solo      1.434001
Pct.1     1.

In [26]:
all_tables[2018]['nwe'].keys()
#'logo', 'name', 'conf'

dict_keys(['logo', 'name', 'conf', 'passing', 'rushing_and_receiving', 'returns', 'kicking', 'defense', 'scoring'])

In [27]:
confs = ['AFC','NFC']
divs = list(set([all_tables[GEN_YEAR][t]['conf'][:-9] for t in teams]))
divs

['NFC East',
 'AFC South',
 'NFC West',
 'AFC West',
 'AFC North',
 'NFC South',
 'AFC East',
 'NFC North']

In [28]:
import random
base = {}

base['startingSeason'] = tyear[0]
valid_tids = list(range(len(teams)))

new_teams = []

for i,t1 in enumerate(sorted(list(teams))):
    t = {}
    t['abbrev'] = t1.upper()
    nsplit = all_tables[GEN_YEAR][t1]['name'].split(' ')
    t['region'] = ' '.join(nsplit[:-1])
    t['name'] = nsplit[-1]
    t['imgURL'] = all_tables[GEN_YEAR][t1]['logo']
    t['tid'] = i 
    t['did'] = divs.index(all_tables[GEN_YEAR][t1]['conf'][:9].rstrip())
    t['cid'] = [c in all_tables[GEN_YEAR][t1]['conf'] for c in confs].index(True)
    new_teams.append(t)
print(new_teams)
team_abbbrevs = list(teams)
    
base['teams'] = new_teams
base['gameAttributes'] = []
base['version'] = 33
DIVS = []
for div in divs:
    cid = [c in div for c in confs].index(True)
    did = divs.index(div)
    DIVS.append({'did':did, 'cid':cid, 'name':div})
print(DIVS)
base['gameAttributes'].append({'key':'divs', 'value': DIVS})
base['gameAttributes'].append({'key':'confs', 'value': [{'cid':i, 'name':n } for i,n in enumerate(confs)]})


[{'abbrev': 'ATL', 'region': 'Atlanta', 'name': 'Falcons', 'imgURL': 'https://d2p3bygnnzw9w3.cloudfront.net/req/201907232/tlogo/pfr/atl-2018.png', 'tid': 0, 'did': 5, 'cid': 1}, {'abbrev': 'BUF', 'region': 'Buffalo', 'name': 'Bills', 'imgURL': 'https://d2p3bygnnzw9w3.cloudfront.net/req/201907232/tlogo/pfr/buf-2018.png', 'tid': 1, 'did': 6, 'cid': 0}, {'abbrev': 'CAR', 'region': 'Carolina', 'name': 'Panthers', 'imgURL': 'https://d2p3bygnnzw9w3.cloudfront.net/req/201907232/tlogo/pfr/car-2018.png', 'tid': 2, 'did': 5, 'cid': 1}, {'abbrev': 'CHI', 'region': 'Chicago', 'name': 'Bears', 'imgURL': 'https://d2p3bygnnzw9w3.cloudfront.net/req/201907232/tlogo/pfr/chi-2018.png', 'tid': 3, 'did': 7, 'cid': 1}, {'abbrev': 'CIN', 'region': 'Cincinnati', 'name': 'Bengals', 'imgURL': 'https://d2p3bygnnzw9w3.cloudfront.net/req/201907232/tlogo/pfr/cin-2018.png', 'tid': 4, 'did': 4, 'cid': 0}, {'abbrev': 'CLE', 'region': 'Cleveland', 'name': 'Browns', 'imgURL': 'https://d2p3bygnnzw9w3.cloudfront.net/req/2

In [29]:
confs

['AFC', 'NFC']

In [30]:
'QB', 'OL', 'TE', 'DL', 'S', 'CB', 'WR', 'LB', 'RB', 'P', 'K'
pos_mapping = {
 'LG':'OL',
 'RG':'OL',
 'CB':'CB',
 'LOLB':'LB',
 'RB':'RB',
 'FB':'RB',
 'K':'K',
 'C':'OL',
 'LCB':'CB',
 'RCB':'CB',
 'FS':'S',
 'RDT':'DL',
 'QB':'QB',
 'DL':'DL',
 'MLB':'LB',
 'WR':'WR',
 'S':'S',
 'SS':'S',
 'TE':'TE',
 'DT':'DL',
 'RILB':'LB',
 'T':'OL',
 'LDT':'DL',
 'P':'P',
 'LILB':'LB',
 'ROLB':'LB',
 'LB':'LB',
 'DB':'S',
 'DE':'DL',
 'NT':'DL'
}

In [31]:
if gen_FA > 0:
    player_names_f = player_names + ["Free Agent{}".format(write_roman(i)) for i in range(gen_FA)]
    player_years_f = player_years + [GEN_YEAR for i in range(gen_FA)]
    # scale them down, we don't want a bunch of amazing replacements
    MEAN_S = 0.95
    STD_S = 0.25
    rp_ratings = np.random.multivariate_normal(MEAN_S*replacement_player_mean_r,STD_S*replacement_player_cov_r,size=(gen_FA))
    ratings_f = np.vstack([ratings,rp_ratings])
else:
    player_names_f = player_names
    player_years_f = player_years
    ratings_f = ratings

In [32]:
l2 = [_.lower() for _ in y.columns]
l2[1] = 'stre'
l2[3] = 'endu'
l2[-7] = 'tck'

In [33]:
valid_pos = df.Pos.unique()
clf2 = linear_model.LogisticRegression()
clf2.fit(X,df.Pos)
pred_pos = clf2.predict(Xn)
print(clf2.score(X,df.Pos))

/Users/lkeselma/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lkeselma/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6246191346739792


/Users/lkeselma/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [34]:
import pprint
import copy
gen_player = []
pp = pprint.PrettyPrinter()
for i,name in enumerate(player_names_f):
    py = player_years_f[i]
    name = player_names_f[i].replace('*','').replace('+','')
    if name in ['Team Total','Opp Total']:
        continue
    sname = name.split(' ')
    new_player = {}
    new_player['firstName'] = sname[0]
    new_player['lastName'] = ' '.join(sname[1:])
    
     
    r_vec = {k: ratings_f[i][i2] for i2,k in enumerate(l2)}
    r_vec = {k: int(np.clip(v,0,100)) for k,v in r_vec.items()}
    #for t in ['hgt', 'stre', 'spd', 'jmp', 'endu', 'ins', 'dnk', 'ft', 'fg', 'tp', 'diq', 'oiq', 'drb', 'pss', 'reb']:
    #    if not t in r_vec:
    #        r_vec[t] = 50
    if True:
        try:
            possible_pos = [_.upper() for _ in player_pos[i] if _ != '0.0']
            pposs = []
            for pos in list(possible_pos):
                if '/' in pos:
                    pp = pos.split('/')
                    for ppp in pp:
                        pposs.append(ppp)
                else:
                    pposs.append(pos)
            possible_pos = [pos_mapping[_] for _ in set(pposs)]
            true_pos = list(set([_ for _ in possible_pos if _ in valid_pos]))
            random.shuffle(true_pos)
            r_vec['pos'] = true_pos[0]
            new_player['pos'] =true_pos[0]
            #print(set(true_pos))
        except:
            r_vec['pos'] = pred_pos[i]
            new_player['pos'] = pred_pos[i]
            #print(i,  player_pos[i])
            pass

    new_player['ratings'] = [r_vec]
    age = player_ages[i]
    new_player['born'] = {'year':2019-age,'loc':''}
    new_player['tid'] = team_abbbrevs.index(player_teams[i])

    
    gen_player.append(new_player)


In [35]:
gen_player

[{'firstName': 'Matt',
  'lastName': 'Ryan',
  'pos': 'QB',
  'ratings': [{'hgt': 68,
    'stre': 32,
    'spd': 47,
    'endu': 45,
    'thv': 72,
    'thp': 75,
    'tha': 80,
    'bsc': 49,
    'elu': 31,
    'rtr': 34,
    'hnd': 40,
    'rbk': 41,
    'pbk': 31,
    'pcv': 24,
    'tck': 9,
    'prs': 10,
    'rns': 14,
    'kpw': 18,
    'kac': 19,
    'ppw': 16,
    'pac': 18,
    'pos': 'QB'}],
  'born': {'year': 1986, 'loc': ''},
  'tid': 0},
 {'firstName': 'Matt',
  'lastName': 'Schaub',
  'pos': 'QB',
  'ratings': [{'hgt': 55,
    'stre': 31,
    'spd': 33,
    'endu': 34,
    'thv': 58,
    'thp': 63,
    'tha': 65,
    'bsc': 24,
    'elu': 19,
    'rtr': 13,
    'hnd': 20,
    'rbk': 31,
    'pbk': 27,
    'pcv': 23,
    'tck': 9,
    'prs': 12,
    'rns': 10,
    'kpw': 9,
    'kac': 13,
    'ppw': 11,
    'pac': 13,
    'pos': 'QB'}],
  'born': {'year': 1982, 'loc': ''},
  'tid': 0},
 {'firstName': 'Mohamed',
  'lastName': 'Sanu',
  'pos': 'WR',
  'ratings': [{'hgt': 55

In [36]:
sum(['pos' in _['ratings'][0] for _ in gen_player]),len(gen_player)

(1671, 1671)

In [39]:
base['players'] = gen_player
with open('fbgm_roster_{}.json'.format(tyear[0]),'wt') as fp:
    json.dump(base,fp, sort_keys=True)

In [41]:
len(ratings),len(ratings_f),len(ratings)+gen_FA
gen_player

[{'firstName': 'Matt',
  'lastName': 'Ryan',
  'pos': 'QB',
  'ratings': [{'hgt': 68,
    'stre': 32,
    'spd': 47,
    'endu': 45,
    'thv': 72,
    'thp': 75,
    'tha': 80,
    'bsc': 49,
    'elu': 31,
    'rtr': 34,
    'hnd': 40,
    'rbk': 41,
    'pbk': 31,
    'pcv': 24,
    'tck': 9,
    'prs': 10,
    'rns': 14,
    'kpw': 18,
    'kac': 19,
    'ppw': 16,
    'pac': 18,
    'pos': 'QB'}],
  'born': {'year': 1986, 'loc': ''},
  'tid': 0},
 {'firstName': 'Matt',
  'lastName': 'Schaub',
  'pos': 'QB',
  'ratings': [{'hgt': 55,
    'stre': 31,
    'spd': 33,
    'endu': 34,
    'thv': 58,
    'thp': 63,
    'tha': 65,
    'bsc': 24,
    'elu': 19,
    'rtr': 13,
    'hnd': 20,
    'rbk': 31,
    'pbk': 27,
    'pcv': 23,
    'tck': 9,
    'prs': 12,
    'rns': 10,
    'kpw': 9,
    'kac': 13,
    'ppw': 11,
    'pac': 13,
    'pos': 'QB'}],
  'born': {'year': 1982, 'loc': ''},
  'tid': 0},
 {'firstName': 'Mohamed',
  'lastName': 'Sanu',
  'pos': 'WR',
  'ratings': [{'hgt': 55

In [ ]:
sorted(['S','DL'])

In [ ]:
ppos = [_.upper() for _ in set(sum(player_pos,[])) if _ != '0.0']

In [ ]:
pPos = []
for p in list(ppos):
    if '/' in p:
        for pp in p.split('/'):
            pPos.append(pp)
    else:
        pPos.append(p)
pPos = list(set(pPos))

In [ ]:
df.Pos.unique()